In [ ]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from scipy.spatial.distance import cosine
from tqdm import tqdm

# 설정
n_qubits = 3
shots = 1024
iterations = 20
eta = 0.25  # 회전 피드백 학습률
sim = Aer.get_backend('qasm_simulator')

# 초기 회전값 (RY 각도)
theta = np.random.rand(n_qubits)

# 기록용
intent_vectors = []
similarities = []
chaos_detected = False

# 분포 → [0,1]^n 벡터로 변환
def counts_to_vector(counts, n=n_qubits):
    total = sum(counts.values())
    vec = np.zeros(n)
    for bitstring, count in counts.items():
        prob = count / total
        bits = [int(b) for b in bitstring[::-1]]  # Qiskit은 큐빗 순서가 반대
        vec += np.array(bits) * prob
    return vec

# 회로 실행
def run_circuit(theta):
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i, angle in enumerate(theta):
        qc.ry(angle * np.pi, i)
    qc.measure(range(n_qubits), range(n_qubits))
    result = execute(qc, sim, shots=shots).result()
    return result.get_counts()

# 반복 실행
print("=== 혼란 상태 실험 시작 ===")
for i in tqdm(range(iterations)):
    counts = run_circuit(theta)
    vec = counts_to_vector(counts)
    intent_vectors.append(vec)

    # 이전 분포와의 유사도 비교
    if i > 0:
        sim_val = 1 - cosine(intent_vectors[i], intent_vectors[i - 1])
        similarities.append(sim_val)
    else:
        similarities.append(1.0)

    # 피드백: 각 큐빗에 bias만큼 회전값 수정
    theta += eta * (vec - 0.5)

    # 혼란 판단
    if i >= 5:
        recent = similarities[-5:]
        if all(s < 0.95 for s in recent):
            chaos_detected = True
            print(f"[!] 혼란 상태 감지됨: iteration {i}")
            break

# 결과 출력
print("\n=== 최종 회전 각도 ===")
print(np.round(theta, 4))

print("\n=== 출력 상태 벡터 (마지막 5회) ===")
for v in intent_vectors[-5:]:
    print(np.round(v, 4))

print("\n=== 코사인 유사도 흐름 ===")
print(np.round(similarities, 4))
